In [1]:
import importlib as il
il.invalidate_caches()
glob               =il.import_module('glob')
os                 =il.import_module('os')
sys                =il.import_module('sys')
sys.path.insert(0,'/notebook_dir/writable-workspace/cccs-utilities/data-tools')
sys.path.insert(0,'/notebook_dir/writable-workspace/cccs-utilities/misc-tools')
Thredds_extraction_functions  =il.import_module('Thredds_extraction_functions')
ECCC_IDF_reader               =il.import_module('ECCC_IDF_reader')
SSH_SCP_connector             =il.import_module('SSH-SCP-connector')
xr                 =il.import_module('xarray')
xci                =il.import_module('xclim.indices')

np                 =il.import_module('numpy')
pd                 =il.import_module('pandas')
from tqdm import tqdm
from xclim.ensembles import create_ensemble
from scp import SCPClient
from xclim.ensembles import ensemble_percentiles

2022-02-09 18:33:26,550 - pint.util - WARNING - Redefining 'year' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,552 - pint.util - WARNING - Redefining 'yr' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,553 - pint.util - WARNING - Redefining 'degree_Celsius' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,553 - pint.util - WARNING - Redefining '°C' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,554 - pint.util - WARNING - Redefining 'celsius' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,555 - pint.util - WARNING - Redefining 'degC' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,556 - pint.util - WARNING - Redefining 'degreeC' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,557 - pint.util - WARNING - Redefining 'C' (<class 'pint.definitions.UnitDefinition'>)
2022-02-09 18:33:26,558 - pint.util - WARNING - Redefining 'deg_C' (<class 'pint.definitions.UnitDefinition'>)


In [ ]:
rcps=['rcp26','rcp45','rcp85']

# Load temperature data
ds=[]
for rcp in rcps:
    file_list=Thredds_extraction_functions.threddscall(rcps=rcp,
                                                       indices='tg_mean',
                                                       frequency='YS',
                                                       average=True)
    ds.append(create_ensemble([f.opendap_url() for f in file_list]))
temperature_data=xr.concat(ds,dim='rcp')
#temperature_data=temperature_data.rename_dims({'realization':'model'})
#temperature_data=temperature_data.rename_vars({'realization':'model'})

mod_time_as_int=temperature_data.coords['time'].values.astype('datetime64[Y]').astype(int)+1970

In [3]:
n_rcps=len(temperature_data.coords['rcp'])
n_models=len(temperature_data.coords['realization'])
n_time=len(temperature_data.coords['time'])
n_durations=9
n_return_periods=6
n_vars=2 #raw #s, and 95% confidence ranges

In [4]:
# Load IDF data
IDF_files=glob.glob('/***REMOVED******REMOVED***/IDF-files/IDF_v-3.20_2021_03_26_*/*')
IDF_files
tmp=ECCC_IDF_reader.read_ECCC_IDF(IDF_files[0])
n_stations=len(IDF_files)
station_names=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['name'] for f in IDF_files]
station_ID=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['ID'] for f in IDF_files]
station_latitudes=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['latitude'] for f in IDF_files]
station_longitudes=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['longitude'] for f in IDF_files]

In [5]:
###write dicts and compare to ClimateData.ca
IDF_dict=[ECCC_IDF_reader.read_ECCC_IDF(f) for f in IDF_files[0:6]]
IDF_dict

[{'location': {'ID': '6166418',
   'name': 'PETERBOROUGH_A',
   'latitude': 44.233333333333334,
   'longitude': 78.36666666666666},
  'period': {'start_date': 1971, 'end_date': 2006, 'total_years': 33},
  'IDF_rates':         2 year  5 year  10 year  25 year  50 year  100 year
  5 min     92.0   121.0    140.2    164.4    182.3     200.2
  10 min    68.2    87.7    100.7    117.0    129.1     141.1
  15 min    56.0    73.1     84.5     98.8    109.4     120.0
  30 min    35.9    47.8     55.6     65.5     72.9      80.2
  1 h       22.1    30.1     35.4     42.1     47.1      52.0
  2 h       13.9    19.9     23.9     29.0     32.7      36.4
  6 h        6.4     8.7     10.2     12.2     13.6      15.0
  12 h       3.7     4.9      5.7      6.7      7.5       8.2
  24 h       2.0     2.7      3.1      3.7      4.1       4.5,
  'IDF_rate_confidence':         2 year  5 year  10 year  25 year  50 year  100 year
  5 min     10.3    17.3     23.3     31.5     37.7      43.9
  10 min     6.9

In [6]:
#create new, empty xarray dataset to hold all results
IDF_da=xr.DataArray(np.empty((n_stations,n_durations,n_return_periods,n_rcps,n_models,n_time)),
                         dims=[     'station', 'duration', 'return_period', 'rcp', 'realization', 'time',],
                         coords=dict(station=station_ID,
                                     duration=tmp['IDF_rates'].index.values,
                                     return_period=tmp['IDF_rates'].columns.values,
                                     rcp=rcps,
                                     models=temperature_data.coords['realization'],
                                     time=temperature_data.coords['time'],
                                    ))

IDF_confidence_da=xr.DataArray(np.empty((n_stations,n_durations,n_return_periods)),
                         dims=[     'station', 'duration', 'return_period'],
                         coords=dict(station=station_ID,
                                     duration=tmp['IDF_rates'].index.values,
                                     return_period=tmp['IDF_rates'].columns.values,))

IDF_station_latitude_da=xr.DataArray(np.empty((n_stations)),
                         dims=[     'station'],
                         coords=dict(station=station_ID,))

IDF_station_longitude_da=xr.DataArray(np.empty((n_stations)),
                         dims=[     'station'],
                         coords=dict(station=station_ID,))

In [7]:
for n,f in enumerate(tqdm(IDF_files[0:3])):
    IDF_dict=ECCC_IDF_reader.read_ECCC_IDF(f)
    T=temperature_data['tg_mean'].sel(dict(lat=IDF_dict['location']['latitude'],
                                        lon=IDF_dict['location']['longitude']*-1.),
                                        method='nearest')

100%|██████████| 3/3 [00:00<00:00, 30.49it/s]


In [29]:
#check which station this is and check against CD.ca
print(IDF_dict)

{'location': {'ID': '6128330', 'name': 'TOBERMORY_RCS', 'latitude': 45.233333333333334, 'longitude': 81.63333333333334}, 'period': {'start_date': 1988, 'end_date': 2017, 'total_years': 23}, 'IDF_rates':         2 year  5 year  10 year  25 year  50 year  100 year
5 min     88.8   126.4    151.2    182.7    206.0     229.1
10 min    67.6    99.8    121.1    148.0    168.0     187.9
15 min    56.3    86.8    106.9    132.4    151.3     170.1
30 min    39.0    65.1     82.3    104.1    120.3     136.3
1 h       25.9    44.1     56.1     71.3     82.5      93.7
2 h       16.1    25.9     32.4     40.6     46.8      52.8
6 h        7.1    10.7     13.1     16.0     18.3      20.5
12 h       3.9     5.8      7.0      8.6      9.8      11.0
24 h       2.1     3.0      3.6      4.4      4.9       5.5, 'IDF_rate_confidence':         2 year  5 year  10 year  25 year  50 year  100 year
5 min     15.6    26.3     35.5     47.9     57.3      66.7
10 min    13.4    22.5     30.4     41.0     49.1    

In [7]:
#check against CD.ca
tens = ensemble_percentiles(T, values=(10, 50, 90))
tens.tg_mean_p50.values - 273.15

array([[ 5.7211853 ,  5.77747498,  6.06000671,  6.38125   ,  6.74825439,
         7.14086304,  7.48400269,  7.89568481,  8.08602295,  8.14855347,
         8.16582642,  8.13909302,  8.16234741],
       [ 5.72014771,  5.77700195,  6.06589661,  6.39399109,  6.85067139,
         7.26929626,  7.71907959,  8.02163086,  8.44866333,  8.83550415,
         9.07351074,  9.29619751,  9.36025391],
       [ 5.72033081,  5.78429565,  6.06157837,  6.42858276,  6.89156494,
         7.3247467 ,  7.90810547,  8.4006897 ,  8.96392212,  9.69443665,
        10.42266235, 11.18578491, 11.90474854]])

In [8]:
base_time=int(np.mean((IDF_dict['period']['start_date'],
                       IDF_dict['period']['end_date'])))
print(IDF_dict['period']['start_date'])
print(IDF_dict['period']['end_date'])
print(base_time)
print(mod_time_as_int)
ibase_T=np.argmin(abs(mod_time_as_int-(base_time-15)))

print(ibase_T)

1988
2017
2002
[1951 1961 1971 1981 1991 2001 2011 2021 2031 2041 2051 2061 2071]
4


In [ ]:
dT=T-T[:,:,ibase_T]
dT.values

In [40]:
dT.attrs['units']='delta_degreeC' #change units for CC equation

pr_baseline=xr.DataArray(IDF_dict['IDF_rates'],
                    coords=dict(duration=IDF_da.coords['duration'],
                                return_period=IDF_da.coords['return_period']),
                    attrs={"units":"mm/hour"})
pr_baseline

<xarray.DataArray (duration: 9, return_period: 6)>
array([[ 88.8, 126.4, 151.2, 182.7, 206. , 229.1],
       [ 67.6,  99.8, 121.1, 148. , 168. , 187.9],
       [ 56.3,  86.8, 106.9, 132.4, 151.3, 170.1],
       [ 39. ,  65.1,  82.3, 104.1, 120.3, 136.3],
       [ 25.9,  44.1,  56.1,  71.3,  82.5,  93.7],
       [ 16.1,  25.9,  32.4,  40.6,  46.8,  52.8],
       [  7.1,  10.7,  13.1,  16. ,  18.3,  20.5],
       [  3.9,   5.8,   7. ,   8.6,   9.8,  11. ],
       [  2.1,   3. ,   3.6,   4.4,   4.9,   5.5]])
Coordinates:
  * duration       (duration) object '5 min' '10 min' '15 min' ... '12 h' '24 h'
  * return_period  (return_period) object '2 year' '5 year' ... '100 year'
Attributes:
    units:    mm/hour

In [ ]:
pr_future=xci.clausius_clapeyron_scaled_precipitation(dT,pr_baseline)
pr_future.values

In [43]:
IDF_da.loc[IDF_dict['location']['ID'],:,:,:,:,:] = pr_future

In [44]:
IDF_confidence_da.loc[IDF_dict['location']['ID'],:,:] = IDF_dict["IDF_rate_confidence"]

IDF_station_latitude_da.loc[IDF_dict['location']['ID']] = IDF_dict['location']['latitude']
IDF_station_longitude_da.loc[IDF_dict['location']['ID']] = IDF_dict['location']['longitude']

In [45]:
#used lat and lon to align with other ds on CD.ca
IDF_df=xr.Dataset(dict(IDF_data=IDF_da,
                       IDF_confidence=IDF_confidence_da,
                       lat=IDF_station_latitude_da,
                       lon=IDF_station_longitude_da,))